## Intro to Tensorflow for AI - Week 4
---
This notebook contains exercises for week 4 of the [Coursera Course](https://www.coursera.org/learn/introduction-tensorflow/lecture/zOIps/a-conversation-with-andrew-ng) Introduction into Tensorflow for AI Development

So far we are still working with image classifications.  In this case we are looking at perhaps some more complicated data sets (less ideal images).

---

Tensorflow's image generator has a really cool feature by which it can scan directory tree structure to identify training and validation data sets and image labels by sub directories within each training and validation set. Neat!

In [3]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Generating our data sets
train_dir = '/home/Documets/Python'
validation_dir = '/home/Documents/Python'

train_datagen = ImageDataGenerator(rescale=1./255) # Buidling in a normalization
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300,300), # Resizing our images for us
    batch_size=128, # This is importat to keep track of
    class_mode='binary'
)

In [8]:
# Neural Network for Horses vs Humans

# Our images are larger so we need more convolution and pooling.
# Since these images are in color we have a 3 in our third dimension
# for our initial convolution (red, green, blue bits).

# Sigmoid is a good activation function for binary classifiers.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                          input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Binary, only one neuron
])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flat

Now that we have built a fairly impressive model, reducing our input from over 900,000 vector length to just 78,400, we need to compile it.  We will use slightly different optimizers and loss functions here.

In [9]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['acc'])

Because we will be using a generator to build our training and test data sets, we use a fitting function that expects a generator object.

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8, # This is determined by the batch size of your train_generator and the size of the data
    epochs=15,
    validation_data=validation_generator,
    validation_steps=8, # this too is determined by batch size in generators
    verbose=2
)